In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:

# Here it is - see the base_url
# You need to do this one time on your computer
!ollama pull llama3.2

from openai import OpenAI

MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "What is 2 + 2?"}]
)
print(response.choices[0].message.content)


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


The answer to the classic math problem is:

2 + 2 = 4


In [5]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        self.title = ""
        self.text = ""
        self.links = []

        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raises HTTPError for bad responses
            self.body = response.content
            soup = BeautifulSoup(self.body, 'html.parser')

            self.title = soup.title.string if soup.title else "No title found"

            if soup.body:
                for irrelevant in soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                self.text = ""

            links = [link.get('href') for link in soup.find_all('a')]
            self.links = [link for link in links if link]

        except Exception as e:
            print(f"Error fetching or parsing {url}: {e}")

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [6]:
ed = Website("https://edwarddonner.com")
print(ed.links)

['https://edwarddonner.com/', 'https://edwarddonner.com/connect-four/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/', 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/12/21/llm-resources-superd

In [7]:
link_system_prompt="You are provided with a list of links found on a webpage. \
         You are able to decide which of the links would be most relevant to include in a brochure about the company, \
                  such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt+="You should respond in JSON as this example:"
link_system_prompt+="""
{ 
      "links":[
      {"type":"about page", "url": "https://full.url/goes/here/about"},
      {"type": "careers page": "url": "https://another.full.url/careers"}

      ]


}
"""


In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.          You are able to decide which of the links would be most relevant to include in a brochure about the company,                   such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as this example:
{ 
      "links":[
      {"type":"about page", "url": "https://full.url/goes/here/about"},
      {"type": "careers page": "url": "https://another.full.url/careers"}

      ]


}



In [9]:
def get_links_user_prompt(website):
         user_prompt=f"Here is the list of links on the website of {website.url} -"
         user_prompt+="please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
                  Do not include Terms of Service, Privacy, email links.\n"
         user_prompt+="Links (some might be relative links):\n"
         user_prompt+="\n".join(website.links)
         return user_prompt



In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com -please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format.                   Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.

In [11]:
def get_links(url):
         website=Website(url)
         response=openai.chat.completions.create(
                 model=MODEL,
                 messages=[
                         {"role":"system","content":link_system_prompt},
                         {"role":"user","content":get_links_user_prompt(website)}

                 ],
                 response_format={"type":"json_object"}
         )
         result=response.choices[0].message.content
         return json.loads(result)

In [12]:

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nvidia/parakeet-tdt-0.6b-v2',
 '/nari-labs/Dia-1.6B',
 '/ACE-Step/ACE-Step-v1-3.5B',
 '/Lightricks/LTX-Video',
 '/lodestones/Chroma',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/smolagents/computer-agent',
 '/spaces/ByteDance/DreamO',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/ACE-Step/ACE-Step',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/DMindAI/DMind_Benchmark',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/PrimeIntellect/INTELLECT-2-RL-Dataset',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/d

In [13]:
def get_all_details(url):
         result="Landing page:\n"
         result+=Website(url).get_contents()
         links=get_links(url)
         print("Found links:",links)
         for link in links["links"]:
                 result+=f"\n\n{link['type']}\n"
                 result+=Website(link["url"]).get_contents()
         return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'About/Hugging Face', 'url': 'https://huggingface.co/'}, {'type': 'Careers/Hugging Face', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog/Hugging Face', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub/Hugging Face', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter/Hugging Face', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn/Hugging Face', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Brand Page/Hugging Face', 'url': 'https://huggingface.co/brand'}, {'type': 'GitHub Repository/Parakeet-TDT-0.6B', 'url': 'https://github.com/nvidia/parakeet-tdt-0.6b-v2'}]}
Error fetching or parsing https://github.com/nvidia/parakeet-tdt-0.6b-v2: 404 Client Error: Not Found for url: https://github.com/nvidia/parakeet-tdt-0.6b-v2
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
         user_prompt = ""
         user_prompt+=f"You are looking at a company called: {company_name}\n"
         user_prompt+=f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown. \n"
         user_prompt+=get_all_details(url)
         user_prompt=user_prompt[:5_000]
         return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog', 'url': 'https://blog.huggingface.co/'}, {'type': 'Discussions Forum', 'url': 'https://discuss.huggingface.co/'}, {'type': 'Status Page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub Repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter Handle', 'url': 'https://twitter.com/huggingface'}, {'type': '.LinkedIn Company Profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Error fetching or parsing https://blog.huggingface.co/: HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002148A753920>: Failed to resolve 'blog.huggingface.co' ([Errno 11001] getaddrinfo failed)"))


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown. \nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnvidia/parakeet-tdt-0.6b-v2\nUpdated\nabout 1 hour ago\n•\n162k\n•\n858\nnari-labs/Dia-1.6B\nUpdated\n2 days ago\n•\n172k\n•\n2.17k\nACE-Step/ACE-Step-v1-3.5B\nUpdated\n3 days ago\n•\n422\nLightricks/LTX-Video\nUpdated\nabout 20 hours ago\n•\n288k\n•\n1.48k\nlodestones/Chroma\nUpdated\n2 days ago\n•\n533\nBrowse 1M+ models\nSpaces\nRunning\n6.57k\n6.57k\nDeepSite\n🐳\nGenerate any application 

In [18]:
def create_brochure(company_name, url):
         response=openai.chat.completions.create(
                 model=MODEL,
                 messages=[
                         {"role":"system","content":system_prompt},
                         {"role":"user","content":get_brochure_user_prompt(company_name, url)}

                 ],
         )

         result=response.choices[0].message.content
         display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")


Found links: {'links': [{'type': 'About Us', 'url': 'https://huggingface.co'}, {'type': 'Company/Enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'Careers/Jobs', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog', 'url': 'https://blog.huggingface.co'}, {'type': 'Discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'Status/Support', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub Repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'FAQs', 'url': 'https://huggingface.co/docs'}, {'type': 'Models', 'url': 'https://huggingface.co/models'}]}
Error fetching or parsing https://blog.huggingface.co: HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002148B015CD0>: Fa

# Hugging Face Brochure

## Introduction

Hugging Face is the premier platform for the machine learning community, empowering individuals and organizations to build, collaborate, and accelerate their AI applications.

### Our Mission

To create a collaborative environment where researchers, developers, and businesses can come together to push the boundaries of Artificial Intelligence.

### Who We Are

Hugging Face is driven by our passion for the potential of AI to transform industries and improve lives. Our community-driven platform encourages experimentation, innovation, and collaboration across various modalities, including text, image, video, audio, and 3D.

## Our Core Offerings

*   **Models**: Browse over 1 million pre-trained models optimized for various applications.
*   **Datasets**: Access and share a vast array of datasets to augment your AI projects.
*   **Spaces**: Publish and run unlimited public models, datasets, and applications with ease.
*   **Compute**: Get access to optimized inference endpoints and GPU deployment capabilities.

### Key Features

*   **Community Engagement**: Join over 50,000 organizations using our platform, including Meta, Amazon, Google, Intel, Microsoft, Grammarly, and Writer.
*   **Accelerate Your ML Journey**: Utilize our compute services to train your models faster with multi-GPU or TPU support.

### Transforming Industries

Hugging Face is revolutionizing the field of AI and enabling businesses to build, deploy, and maintain their own machine learning models.

## Stay Ahead of the Curve

Join our growing community today! Sign up for a free account and start exploring our platform:

[Sign up now](https://huggingface.co/login)

## Career Opportunities

Ready to join the Hugging Face team? Explore our current job openings:

[ careers ](https://careers.huggingface.co/)

## Get in Touch

Want to connect with us directly? Follow us on social media or visit our website:

[ Website ](https://huggingface.co/)
[@ HuggingFace On Twitter](https://twitter.com/HuggingFace)
[ LinkedIn Hugging Face ](https://www.linkedin.com/company/hugging-face-ai/)

---

Stay up to date with our latest publications, research papers, and insights into the world of AI! Follow us:

[@ PaperHuggingOn GitHub](https://github.com/HuggingFace?)
[@ paper_hugging_on Twitter](/)
[ The Official Hugging Face Blog ](https://blog.huggingface.co/)

---

#

In [20]:
def stream_brochure(company_name, url):
         stream=openai.chat.completions.create(
                 model=MODEL,
                 messages=[
                         {"role":"system","content":system_prompt},
                         {"role":"user","content":get_brochure_user_prompt(company_name,url)}
                 ],
                 stream=True
                 
         )

         response= ""
         display_handle=display(Markdown(""), display_id=True)
         for chunk in stream:
                  response+=chunk.choices[0].delta.content or ''
                  response=response.replace("```","").replace("markdown","")
                  update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
brochure_data=stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'Company Page', 'url': 'https://huggingface.co/'}, {'type': 'About Page', 'url': 'https://huggingface.co/blog'}, {'type': 'Careers Page', 'url': 'https://ai.huggingface.co/ jobs'}, {'type': 'Products', 'url': 'https://huggingface.co/models'}, {'type': 'Documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'Blog', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub Repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter Handle', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn Company Page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Error fetching or parsing https://ai.huggingface.co/ jobs: HTTPSConnectionPool(host='ai.huggingface.co', port=443): Max retries exceeded with url: /%20jobs (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002148A1ACE60>: Failed to resolve 'ai.huggingface.co' ([Errno 11001] getaddrinfo failed)"))


**Hugging Face Brochure**
=========================

**Welcome to Hugging Face**
---------------------------

We're the AI community building the future. Our platform is where the machine learning community collaborates on models, datasets, and applications.

**What We Do**
-----------------

* **Build, Discover, and Collaborate**: Host and collaborate on unlimited public models, datasets, and applications.
* **Accelerate Your ML**: Leverage our Hugging Face Open source stack to move faster with AI.
* **Explore All Modalities**: Text, image, video, audio or 3D - we support it all.

**Community Highlights**
-------------------------

*   **1M+ Models**: Browse from a vast library of pre-trained models and create your own.
*   **10k+ Datasets**: Access and share datasets for any ML tasks.
*   **50,000+ Organizations**: Join the ranks of companies like Meta, Google, Amazon, Intel, and Microsoft using our platform.

**Our Mission**
----------------

We're building the foundation of ML tooling with the community. Our open-source projects include:

*   Transformers
*   Diffusers
*   Safetensors
*   Tokenizers
*   TRL (Train Transformed Language Models)
*   Transformers.js
*   Smolagents
*   PEFT

**Join Us**
--------------

Sign up to accelerate your ML and contribute to our mission.

[Get Started](Link to sign-up)

### Culture at Hugging Face
------------------------------------

We prioritize a collaborative, inclusive culture that empowers everyone to participate.

### Opportunities
---------------

We offer various roles across data science, software engineering, sales, and more. Explore our open positions now [here](link).

### Careers
---------

[Learn More](Link to careers)

*   **Jobs**: Stay updated on openings at Hugging Face.
*   **Internships**: Apply for internships to get hands-on experience.
*   **Events**: Attending Webinars, meetups and conferences

### About Us
--------------

[Read Our Story](link to about us page)

In [24]:
def enhance_brochure(brochure_data):
    if brochure_data is None:
        brochure_data = {}

    if "Features and Services" not in brochure_data or brochure_data["Features and Services"] is None:
        brochure_data["Features and Services"] = {}

    brochure_data["Features and Services"].update({
        "Datasets": (
            "Access thousands of public datasets for tasks like text classification, translation, "
            "image generation, audio processing, and more, all integrated with the Hugging Face Hub."
        ),
        "Spaces": (
            "Host and demo ML apps using Gradio or Streamlit. Spaces let developers build interactive showcases."
        ),
        "Transformers": (
            "Access pre-trained models like BERT, GPT, RoBERTa, and more using Hugging Face Transformers library."
        ),
        "Inference API": (
            "Easily deploy any model to production via the Inference API with minimal setup."
        ),
        "Community": (
            "Join researchers and developers around the world. Contribute, discuss, and share ideas."
        )
    })
    return brochure_data


In [25]:
brochure_data = {}
brochure_data = enhance_brochure(brochure_data)
print(brochure_data)


{'Features and Services': {'Datasets': 'Access thousands of public datasets for tasks like text classification, translation, image generation, audio processing, and more, all integrated with the Hugging Face Hub.', 'Spaces': 'Host and demo ML apps using Gradio or Streamlit. Spaces let developers build interactive showcases.', 'Transformers': 'Access pre-trained models like BERT, GPT, RoBERTa, and more using Hugging Face Transformers library.', 'Inference API': 'Easily deploy any model to production via the Inference API with minimal setup.', 'Community': 'Join researchers and developers around the world. Contribute, discuss, and share ideas.'}}


In [26]:
def export_to_markdown(brochure_data, filename="brochure.md"):
    with open(filename, "w", encoding="utf-8") as file:
        for section, content in brochure_data.items():
            file.write(f"# {section}\n\n")
            if isinstance(content, dict):
                for sub_section, detail in content.items():
                    file.write(f"## {sub_section}\n\n{detail}\n\n")
            else:
                file.write(f"{content}\n\n")


In [28]:
print(export_to_markdown(brochure_data))


None


In [30]:
!pip install markdown

In [31]:
import markdown

def export_to_html(markdown_file="brochure.md", html_file="brochure.html"):
    with open(markdown_file, "r", encoding="utf-8") as md_file:
        md_text = md_file.read()
        html = markdown.markdown(md_text)
    with open(html_file, "w", encoding="utf-8") as file:
        file.write(html)


In [32]:
export_to_html()


In [33]:
!pip install markdown2 pdfkit


   ---------------------------------------- 2/2 [markdown2]



In [40]:
!pip install wkhtmltopdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wkhtmltopdf: filename=wkhtmltopdf-0.2-py3-none-any.whl size=11216 sha256=e396aced6f831aa25e014f448b13f34dcf67b1dc9b59834a0c4ee1d7005da29a
  Stored in directory: c:\users\sri lakshmi prasanna\appdata\local\pip\cache\wheels\d7\df\5a\6a89a343df16c425adc9591b782d1250ee0a25df08225f81c7
Successfully built wkhtmltopdf


  DEPRECATION: Building 'wkhtmltopdf' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wkhtmltopdf'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [55]:
import markdown2
import pdfkit
import os

def export_to_pdf(markdown_file="brochure.md", pdf_file="brochure.pdf", wkhtmltopdf_path=None):
    if not os.path.exists(markdown_file):
        print(f"Error: Markdown file '{markdown_file}' not found.")
        return False

    try:
        with open(markdown_file, "r", encoding="utf-8") as md_file:
            markdown_content = md_file.read()

        html = markdown2.markdown(markdown_content)

        # Provide wkhtmltopdf path if passed
        if wkhtmltopdf_path:
            config = pdfkit.configuration(wkhtmltopdf=wkhtmltopdf_path)
            pdfkit.from_string(html, pdf_file, configuration=config)
        else:
            pdfkit.from_string(html, pdf_file)

        if os.path.exists(pdf_file):
            print(f"✅ PDF successfully created: {pdf_file}")
            return True
        else:
            print(f"❌ Error: PDF file '{pdf_file}' was not created.")
            return False

    except Exception as e:
        print(f"❌ Error during PDF conversion: {str(e)}")
        return False


In [58]:
wkhtmltopdf_path = r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe"



In [59]:
export_to_pdf("brochure.md", "brochure.pdf", wkhtmltopdf_path)

✅ PDF successfully created: brochure.pdf


True

In [60]:
!pip install transformers

In [61]:
def read_markdown(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


In [63]:
!ollama pull llama3.2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [70]:

# Here it is - see the base_url
# You need to do this one time on your computer
!ollama pull llama3.2

from openai import OpenAI

MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "What is 2 + 2?"}]
)
print(response.choices[0].message.content)


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling mani

2 + 2 = 4


In [71]:

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It's lovely to meet you! I'm thrilled to be your first conversational partner. How are you doing today? Is there something specific you'd like to chat about or ask for help with? I'm all ears (or rather, all text) and here to assist you!


In [76]:



# Step 2: Translate using local LLaMA 3
def translate_to_spanish(content):
    prompt = f"Translate the following Markdown brochure to Spanish while keeping all formatting (like #, **, etc.) intact:\n\n{content}"
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a professional translator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4
    )
    return response.choices[0].message.content



In [77]:

# Step 3: Save translated output
def save_translation(content, output_path="brochure_es.md"):
    with open(output_path, "w", encoding="utf-8") as file:
        file.write(content)
    print(f"✅ Translated brochure saved to {output_path}")

In [78]:

# Step 4: Run the process
if __name__ == "__main__":
    brochure = read_markdown("brochure.md")
    translated = translate_to_spanish(brochure)
    save_translation(translated)


✅ Translated brochure saved to brochure_es.md


In [ ]:
import shutil

def zip_exports():
    shutil.make_archive("huggingface_brochure", "zip", ".", ".", verbose=1)
    
# Call the function to create the zip
zip_exports()  
# all packages export function